![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_CamemBertForSequenceClassification.ipynb)

# Import OpenVINO CamemBertForSequenceClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting CamemBertForSequenceClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for CamemBertForSequenceClassification from CamemBertForSequenceClassification and they have to be in `Text Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [antoinelouis/crossencoder-camembert-large-mmarcoFR](https://huggingface.co/antoinelouis/crossencoder-camembert-large-mmarcoFR) model from HuggingFace as an example and load it as a `OVModelForSequenceClassification`, representing an OpenVINO model.
- In addition to the OVModelForSequenceClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

MODEL_NAME = "astrosbd/french_emotion_camembert"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/astrosbd/french_emotion_camembert/tokenizer_config.json',
 'ov_models/astrosbd/french_emotion_camembert/special_tokens_map.json',
 'ov_models/astrosbd/french_emotion_camembert/sentencepiece.bpe.model',
 'ov_models/astrosbd/french_emotion_camembert/added_tokens.json',
 'ov_models/astrosbd/french_emotion_camembert/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 435688
-rw-r--r-- 1 root root      1027 Jun 21 02:37 config.json
-rw-r--r-- 1 root root 442510536 Jun 21 02:37 openvino_model.bin
-rw-r--r-- 1 root root    377720 Jun 21 02:37 openvino_model.xml
-rw-r--r-- 1 root root    810912 Jun 21 02:37 sentencepiece.bpe.model
-rw-r--r-- 1 root root      1058 Jun 21 02:37 special_tokens_map.json
-rw-r--r-- 1 root root      1813 Jun 21 02:37 tokenizer_config.json
-rw-r--r-- 1 root root   2421253 Jun 21 02:37 tokenizer.json


- We need to move `sentencepiece.bpe.model` from the tokenizer to assets folder which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [5]:
!mkdir -p {EXPORT_PATH}/assets

labels = ov_model.config.id2label
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [7]:
!ls -lR {EXPORT_PATH}

ov_models/astrosbd/french_emotion_camembert:
total 434900
drwxr-xr-x 2 root root      4096 Jun 21 02:42 assets
-rw-r--r-- 1 root root      1027 Jun 21 02:37 config.json
-rw-r--r-- 1 root root 442510536 Jun 21 02:37 openvino_model.bin
-rw-r--r-- 1 root root    377720 Jun 21 02:37 openvino_model.xml
-rw-r--r-- 1 root root      1058 Jun 21 02:37 special_tokens_map.json
-rw-r--r-- 1 root root      1813 Jun 21 02:37 tokenizer_config.json
-rw-r--r-- 1 root root   2421253 Jun 21 02:37 tokenizer.json

ov_models/astrosbd/french_emotion_camembert/assets:
total 796
-rw-r--r-- 1 root root     35 Jun 21 02:42 labels.txt
-rw-r--r-- 1 root root 810912 Jun 21 02:37 sentencepiece.bpe.model


- Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save CamemBertForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 38.3 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `CamemBertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [10]:
from sparknlp.annotator import CamemBertForSequenceClassification

sequenceClassifier = CamemBertForSequenceClassification.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your CamemBertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {EXPORT_PATH}_spark_nlp_onnx

total 433380
-rw-r--r-- 1 root root 442956119 Jun 21 02:46 camembert_classification_openvino
-rw-r--r-- 1 root root    810912 Jun 21 02:46 camembert_spp
drwxr-xr-x 3 root root      4096 Jun 21 02:45 fields
drwxr-xr-x 2 root root      4096 Jun 21 02:45 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForSequenceClassification model 😊

In [14]:
sequenceClassifier_loaded = CamemBertForSequenceClassification.load(f"./{EXPORT_PATH}_spark_nlp_onnx") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
sequenceClassifier_loaded.getClasses()

['surprise', 'joy', 'anger', 'neutral', 'sad', 'fear']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

example = spark.createDataFrame([
    ["Je n'arrive pas à croire ce qui vient de se passer !"],
    ["Je suis tellement heureux aujourd’hui !"],
    ["Ça m’énerve vraiment quand on me parle comme ça !"],
    ["Aujourd’hui est un jour comme un autre."],
    ["Je me sens tellement triste en ce moment..."],
    ["J’ai peur de ce qui pourrait arriver demain."],
    ["C’était incroyable, je ne m’y attendais pas du tout !"],
    ["J’adore écouter de la musique, ça me rend heureux."],
    ["Je ne supporte pas d’être ignoré !"],
    ["Rien de spécial aujourd’hui, juste une journée normale."],
    ["Mon chat est mort hier, je suis dévasté."],
    ["J’ai entendu un bruit étrange dans la maison, ça me fait peur."]
]).toDF("text")

model = pipeline.fit(example)
result = model.transform(example)

result.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------+----------+
|text                                                          |result    |
+--------------------------------------------------------------+----------+
|Je n'arrive pas à croire ce qui vient de se passer !          |[surprise]|
|Je suis tellement heureux aujourd’hui !                       |[joy]     |
|Ça m’énerve vraiment quand on me parle comme ça !             |[anger]   |
|Aujourd’hui est un jour comme un autre.                       |[neutral] |
|Je me sens tellement triste en ce moment...                   |[sad]     |
|J’ai peur de ce qui pourrait arriver demain.                  |[fear]    |
|C’était incroyable, je ne m’y attendais pas du tout !         |[surprise]|
|J’adore écouter de la musique, ça me rend heureux.            |[joy]     |
|Je ne supporte pas d’être ignoré !                            |[anger]   |
|Rien de spécial aujourd’hui, juste une journée normale.       |[neutral] |
|Mon chat es

That's it! You can now go wild and use hundreds of `CamemBertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
